## Using Cross Validation

In this exercise, you will use cross-validation to optimize parameters for a regression model.

### Prepare the Data

First, import the libraries you will need and prepare the training and test data:

In [1]:
# Import Spark SQL and Spark ML libraries
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

# Load the source data
wkdir ='file:///mnt/c/Users/Adura/Google Drive/Projects/Jupyter/SparkMs/data/'
csv = spark.read.csv(wkdir + 'flights.csv', inferSchema=True, header=True)

# Select features and label
data = csv.select("DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay", col("ArrDelay").alias("label"))

# Split the data
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")

### Define the Pipeline
Now define a pipeline that creates a feature vector and trains a regression model

In [2]:
# Define the pipeline
assembler = VectorAssembler(inputCols = ["DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay"], outputCol="features")
lr = LinearRegression(labelCol="label",featuresCol="features")
pipeline = Pipeline(stages=[assembler, lr])

### Tune Parameters
You can tune parameters to find the best model for your data. To do this you can use the  **CrossValidator** class to evaluate each combination of parameters defined in a **ParameterGrid** against multiple *folds* of the data split into training and validation datasets, in order to find the best performing parameters. Note that this can take a long time to run because every parameter combination is tried multiple times.

In [3]:
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.3, 0.01]).addGrid(lr.maxIter, [10, 5]).build()
cv = CrossValidator(estimator=pipeline, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid, numFolds=2)

model = cv.fit(train)

### Test the Model
Now you're ready to apply the model to the test data.

In [4]:
prediction = model.transform(test)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show()

+--------------------+-------------------+---------+
|            features|         prediction|trueLabel|
+--------------------+-------------------+---------+
|[1.0,1.0,10140.0,...| -7.678267166557415|      -11|
|[1.0,1.0,10140.0,...| -6.875755651792705|      -11|
|[1.0,1.0,10140.0,...| 17.229042212886508|       19|
|[1.0,1.0,10140.0,...| 17.229042212886508|       23|
|[1.0,1.0,10140.0,...|  31.29017430061605|       41|
|[1.0,1.0,10140.0,...| -7.887849859507044|       -8|
|[1.0,1.0,10140.0,...| -1.861650393337241|       -1|
|[1.0,1.0,10140.0,...|0.14708276205269355|        6|
|[1.0,1.0,10140.0,...|-13.915454349316734|      -19|
|[1.0,1.0,10140.0,...| -7.889254883146931|       -1|
|[1.0,1.0,10140.0,...| -4.876155150062028|       -9|
|[1.0,1.0,10140.0,...| -9.120298608076805|       -8|
|[1.0,1.0,10140.0,...| -4.244354007543443|       -2|
|[1.0,1.0,10140.0,...| -7.258156252448288|        1|
|[1.0,1.0,10140.0,...| -6.253789674753321|       -8|
|[1.0,1.0,10140.0,...| 0.5650861407019918|    

### Examine the Predicted and Actual Values
You can plot the predicted values against the actual values to see how accurately the model has predicted. In a perfect model, the resulting scatter plot should form a perfect diagonal line with each predicted value being identical to the actual value - in practice, some variance is to be expected.
Run the cells below to create a temporary table from the **predicted** DataFrame and then retrieve the predicted and actual label values using SQL. You can then display the results as a scatter plot, specifying **-** as the function to show the unaggregated values.

In [5]:
predicted.createOrReplaceTempView("regressionPredictions")

In [6]:
%%sql
SELECT trueLabel, prediction FROM regressionPredictions

ERROR:root:Cell magic `%%sql` not found.


### Retrieve the Root Mean Square Error (RMSE)
There are a number of metrics used to measure the variance between predicted and actual values. Of these, the root mean square error (RMSE) is a commonly used value that is measured in the same units as the prediced and actual values - so in this case, the RMSE indicates the average number of minutes between predicted and actual flight delay values. You can use the **RegressionEvaluator** class to retrieve the RMSE.


In [8]:
evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction)
print("Root Mean Square Error (RMSE):", rmse)


Root Mean Square Error (RMSE): 13.213645496741456
